In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration
import torch
import pandas as pd
import glob
from read_csv_gz import read_csv_gz
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
ed_diagnosis_df = read_csv_gz("ed_data/diagnosis.csv.gz")
ed_edstays_df = read_csv_gz("ed_data/edstays.csv.gz")
ed_medrecon_df = read_csv_gz("ed_data/medrecon.csv.gz")
ed_pyxis_df = read_csv_gz("ed_data/pyxis.csv.gz")
ed_triage_df = read_csv_gz("ed_data/triage.csv.gz")
ed_vitalsign_df = read_csv_gz("ed_data/vitalsign.csv.gz")


In [3]:
# def text_gen(x):
#     return f"You are a medical expert assisting in an Emergency Department (ED). Your task is to assess the **diagnostic complexity** of the following chief complaint on a scale of 1-10:\n- **1-3 (Low Complexity):** Easily diagnosable, requires minimal tests.\n- **4-6 (Moderate Complexity):** Needs some testing, single specialty.\n- **7-10 (High Complexity):** Requires multiple tests, possible admission, specialist consults.\nChief Complaint: **{x}**\nPlease only return an integer of Complexity Score (1-10):"

In [3]:
def text_gen(x):
    return f"The patient is suffering from {x}"

In [4]:
print(ed_triage_df.shape[0])

425087


In [5]:
def converter(df, m):
    ans = []
    temp = []
    for i in range(df.shape[0]):
        text = text_gen(df.iloc[i])
        temp.append(text)
        if (i + 1) % m == 0:
            if len(temp) != 0 :
                ans.append(temp)
            temp = []
    if len(temp) != 0:
        ans.append(temp)
    return ans

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [47]:
def indefence_prompt(batch_lst, tokenizer, model, device="cuda" if torch.cuda.is_available() else "cpu"):
    ans = []
    for batch in tqdm(batch_lst):
        bs = len(batch)
        inputs = tokenizer(batch, padding=True, return_tensors="pt").to(device)
        with torch.no_grad():
            output_sequences = model.encoder(**inputs)
        for i in range(bs):
            ans.append(output_sequences.last_hidden_state.mean(dim=1)[i, :].cpu().numpy())

    return pd.Series(ans)
        
        


In [48]:
lst_prompt = converter(ed_triage_df["chiefcomplaint"].head(234), 10)

In [49]:
model_name = "t5-base"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)
model = T5ForConditionalGeneration.from_pretrained(model_name)
# tokenizer.pad_token = tokenizer.eos_token

In [50]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [51]:
res = indefence_prompt(lst_prompt, tokenizer, model, device="cuda" if torch.cuda.is_available() else "cpu")

100%|██████████| 24/24 [00:01<00:00, 19.43it/s]


In [54]:
print(res.iloc[0].shape)
print(res.shape)

(768,)
(234,)
